<h1>Validação Cruzada <h1>

In [24]:
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

Validação cruzada se refere a maneira que serão divididos os dados de treino e teste do nosso modelo de predição, essa divisão geralmente é realizada várias vezes onde a medida de score do modelo é calculada realizadando uma media de todas as possibilidades modeladas. Essa tecnica geralmente requer um poder computacional bem relevante dependendo do número de subdivisões utilizados nos métodos, que podem ser:

<ui>
<li> K-Fold (K Dobras)
<li> Stratified K-Fold
<li> Leave One Out
<ui>

<h2>K-Fold<h2>

Esse metódo consiste em separar os dados em $K$ subdivisões iguais, sendo que uma das subdivisões (Fold) será guardado para teste e as outras $(K-1)$ serão para treino, depois de realizar esse primeiro modelo e guardar sua pontuação ($R^2$), serão gerados outros $(K-1)$ modelos apenas mudando qual Fold será usada para teste e guardando sua pontuação para chegar em uma média final de desempenho. É recomendado usar um $K$ variando de 5 a 10 Folds.

In [25]:
# Limpeza dos dados, dados de estudantes para serem adimitidos em um exame com base nas suas notas
df_adm = pd.read_csv('Admission_Predict.csv')

df_admiss = (df_adm.drop(['Serial No.'], axis='columns')).copy()

x_admiss = df_admiss.loc[:, df_admiss.columns != 'Chance of Admit ']
y_admiss = df_admiss.loc[:, df_admiss.columns == 'Chance of Admit ']

In [26]:
modelo = LinearRegression()
kfold = KFold(n_splits= 5)
resultado = cross_val_score(modelo, x_admiss, y_admiss, cv = kfold) # separa automaticamente treino e teste

print(resultado.mean())

0.7711794121066354


In [30]:
def modelos_regressao_k_fold (dados_x, dados_y):
    reg = LinearRegression()
    lasso = Lasso()
    ridge = Ridge()
    elastic = ElasticNet(alpha = 1, l1_ratio= 0.5) # Parametros default
    kfold = KFold(n_splits= 10)

    reg_score = cross_val_score(reg, dados_x, dados_y, cv = kfold)
    lasso_score = cross_val_score(lasso, dados_x, dados_y, cv = kfold)
    ridge_score = cross_val_score(ridge, dados_x, dados_y, cv = kfold) 
    elastic_score = cross_val_score(elastic, dados_x, dados_y, cv = kfold) 

    dic_models = {'Linear':reg_score.mean() , 'Lasso':lasso_score.mean(), 'Ridge':ridge_score.mean(), 'Elastic':elastic_score.mean()}

    print("Modelos K-Fold")
    print("Regressão Linear:", reg_score.mean(), "\nLasso:", lasso_score.mean(), "\nRidge:", ridge_score.mean(), "\nElastic Net:", elastic_score.mean())
    print("O melhor modelo foi",max(dic_models, key = dic_models.get),"com valor", max(dic_models.values()))

In [28]:
modelos_regressao_k_fold(x_admiss, y_admiss)

Modelos K-Fold
Regressão Linear: 0.7703825020879949 
Lasso: 0.18277276199791023 
Ridge: 0.7705725482928705 
Elastic Net: 0.49623767436974936
O melhor modelo foi Ridge com valor 0.7705725482928705


<h2>Stratified K-Fold<h2>

Esse método funciona da mesma maneira que o K-Fold, porém ele mantém uma proporção fixa de classes da variável resposta ao longo das subdivisões com a intenção de não deixar algum dos $K$ folds com uma classe apenas, esse tipo de método funciona melhor em algotitimos de classificação. <br>

Por exemplo se temos um conjunto de dados de alunos com 3% sendo aprovados, precisamos que essaa porcentagem de aprovados seja replicada ao longo de cada uma das subdivisões, do contrário poderiam haver folds apenas com dados de alunos reprovados, o que implicaria no modelo não conseguir treinar bem pra prever os aprovados.

<h2>Leave One Out<h2>

A técnica Leave One Out é utilizada quando se trata de pequenos conjuntos de dados, pois é necessário muito poder computacional. Neste caso, cada iteração é composta por um caso da base de dados sendo o conjunto de teste e o restante das observações compondo o conjunto de treinamento. No fim se trata de uma tecnica K-Fold onde o $K$ é a quantidade totais de linhas da tabela. 

<h1>Ajustando parâmetros<h1>

É importante saber alterar os parâmetros dos modelos para chegar em um que apresente um desempenho melhor, no caso da regressão linear elastic net podemos ajustar o valor do $\lambda$ e do coeficiente $L1$ (Lasso) afim de encontrar o melhor modelo.

In [32]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# Definindo valores para testar

valores = {'alpha': [0.1,0.5,1,2,5,10,25,50,100,150,200,300,500,750,1000,1500,2000,3000,5000], 'l1_ratio': [0.02,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}

modelo_aj = ElasticNet()
procura1 = RandomizedSearchCV(estimator= modelo_aj, param_distributions= valores, n_iter= 150, cv = 5, random_state = 15) # cv = K-fold # Essa função testa 150 combinações possíveis
procura = GridSearchCV(estimator= modelo_aj, param_grid= valores,  cv = 5) # Essa função testa todas as combinações possíveis
procura.fit(x_admiss, y_admiss)

print("Melhor Score:", procura.best_score_)
print("Melhor Alpha:", procura.best_estimator_.alpha)
print("Melhor L1 Ratio:", procura.best_estimator_.l1_ratio)

Melhor Score: 0.7408292165331436
Melhor Alpha: 0.1
Melhor L1 Ratio: 0.02
